In [1]:
import pickle as pkl
import pandas as pd
import numpy as np
import json

In [2]:
gens = pkl.load(open('train_generations_try2.pkl', 'rb'))
gens

[{83: [{'Title': ' набор flash point',
    'Price': ' 16000',
    'Count': ' 1',
    'Currency': ' RUB'}],
  368: [{'Title': ' bluetooth-гарнитура',
    'Price': ' 35',
    'Count': ' 1',
    'Currency': ' лари'}],
  456: [{'Title': ' Брюки новые с биркой, 122-128',
    'Price': ' 1000',
    'Count': ' 1',
    'Currency': ' RUB'}],
  655: [{'Title': ' наушники jbl tune 115bt',
    'Price': ' 60',
    'Count': ' 1',
    'Currency': ' лари'},
   {'Title': ' наушники беспроводные redmi buds 3',
    'Price': ' 3',
    'Count': ' 1',
    'Currency': ' лари'}],
  764: [{'Title': ' 14 промакс на 1 трб с аксессуарами',
    'Price': ' 1100',
    'Count': ' 1',
    'Currency': ' USD'}],
  799: [{'Title': ' подсвечник и 2 декор.штучки',
    'Price': ' 10',
    'Count': ' 2',
    'Currency': ' EUR'}],
  852: [{'Title': ' массажёр для тела',
    'Price': ' 30',
    'Count': ' 1',
    'Currency': ' EUR'}],
  879: [{'Title': ' Вельветовая рубашка с поясом',
    'Price': ' 500',
    'Count': ' 1',
   

In [5]:
df = pd.read_csv('../labeled_15k.csv')
df = df[df.Label_gpt == 'valid']
df = df[['Text', 'Title', 'Price', 'Currency', 'Count']].reset_index(drop=True)
df['json'] = pd.Series([[{'Title': row.Title, 'Price': row.Price, 'Currency': row.Currency, 'Count': row.Count}] for row in df.itertuples()])
text_df = df[['Text', 'json']].groupby('Text').sum().reset_index()
text_df['n_bundle'] = text_df.json.apply(len)

single_bundle_mask = text_df.n_bundle == 1
np.random.seed(42)
test_set_index = np.random.choice(np.where(single_bundle_mask)[0], size=300, replace=False)
test_mask = np.zeros(single_bundle_mask.shape, dtype=bool)
test_mask[test_set_index] = True

# text_df, text_df_test = text_df[~test_mask], text_df[test_mask]

# text_df, text_df_test = train_test_split(text_df, random_state=42, test_size=0.1)
print(len(text_df))
text_df.head()

9903


,Text,json,n_bundle
0,! новая. хлопковая рубашка оверсайз с приятной...,"[{'Title': 'хлопковая рубашка оверсайз', 'Pric...",1
1,!!! ПОМИДОРКИ !!! в 3л Банке.... по 250 руб. К...,"[{'Title': 'Икра КРАСНАЯ 140 гр. банка', 'Pric...",3
2,"""конфетный мячик"" 🍬🏀 цена €7😻 бесплатная доста...","[{'Title': 'конфетный мячик', 'Price': '7', 'C...",1
3,# ТРИКО Распродажа без выбораМОДЕЛЬ ТКАНЬ ТРИК...,"[{'Title': 'Трико, двух нитки качества супер',...",1
4,# ТРИКО Распродажа без выбораМОДЕЛЬ ТКАНЬ ТРИК...,[{'Title': 'Суперкачественный трикотаж из двух...,1


In [6]:
print(text_df.loc[56, 'Text'])

#БНИ_ПродамБУ_Калининград 
#БНИ_Доставка 
#БНИ_АвитоДоставка 
#БНИ_existentialism7 
 
Starship Samurai + Starship Samurai: Shattered Alliances - 10к 

Все карты в протекторах и со вставками на русском языке. Жетоны кланов в капсулах. На мехов наклеены символы их силы. База с допом лежит в одной коробке.


In [65]:
table_data = []
for i_gen, checkpoint in enumerate(gens):
  for text_id, output in checkpoint.items():
    outputs_row = {'TextId': text_id, 'Text': text_df.loc[text_id, 'Text'], 'Step': (i_gen + 1) * 300,'Output': json.dumps(output, indent=4, ensure_ascii=False)}
    # outputs_row.update(output)
    table_data.append(outputs_row)
table_df = pd.DataFrame(table_data)
table_df

,TextId,Text,Step,Output
0,56,#БНИ_ПродамБУ_Калининград \n#БНИ_Доставка \n#Б...,300,"[\n {\n ""Title"": "" Starship Samurai ..."
1,125,#БНИ_ПродамБУ_Москва \n#БНИ_Авито \n#БНИ_Почта...,300,"[\n {\n ""Title"": "" Спящие боги"",\n ..."
2,257,#БНИ_ПродамНовая_Москва\n#БНИ_ПродамБУ_Москва\...,300,"[\n {\n ""Title"": "" Vindication Board..."
3,335,#длякухни\n\nэлектрический чайник с функцией у...,300,"[\n {\n ""Title"": "" электрический чай..."
4,347,#продам bluetooth-гарнитуру. подходит как для ...,300,"[\n {\n ""Title"": "" Bluetooth-гарниту..."
...,...,...,...,...
375,1037,ddr3 and hard drive 500gb all 30€ limassol,5700,"[\n {\n ""Title"": "" ddr3 and hard dri..."
376,1041,diablo 4 для ps5 - 100 лари \nтолько до суббот...,5700,"[\n {\n ""Title"": "" diablo 4 для ps5""..."
377,1049,"dodge journey 2009\n2.0 crd sxt \nавтомат, диз...",5700,"[\n {\n ""Title"": "" dodge journey 200..."
378,1056,evga rtx 3070ti xc3 ultra цена 930 лари,5700,"[\n {\n ""Title"": "" evga rtx 3070ti x..."


In [68]:
# table_df.set_index(['TextId', 'Text', 'Step'], inplace=True)
# table_df.groupby(['TextId', 'Text']).apply(lambda x: x.reset_index(drop=True)).reset_index(level=0, drop=True)
table_df.sort_values(['TextId', 'Step'], inplace=True)

table_df.to_csv('outputs_table.csv')

In [69]:
table_df

,TextId,Text,Step,Output
0,56,#БНИ_ПродамБУ_Калининград \n#БНИ_Доставка \n#Б...,300,"[\n {\n ""Title"": "" Starship Samurai ..."
20,56,#БНИ_ПродамБУ_Калининград \n#БНИ_Доставка \n#Б...,600,"[\n {\n ""Title"": "" Starship Samurai ..."
40,56,#БНИ_ПродамБУ_Калининград \n#БНИ_Доставка \n#Б...,900,"[\n {\n ""Title"": "" Starship Samurai ..."
60,56,#БНИ_ПродамБУ_Калининград \n#БНИ_Доставка \n#Б...,1200,"[\n {\n ""Title"": "" Starship Samurai ..."
80,56,#БНИ_ПродамБУ_Калининград \n#БНИ_Доставка \n#Б...,1500,"[\n {\n ""Title"": "" Starship Samurai ..."
...,...,...,...,...
299,1072,god of war: ragnarok (ps4)\n100 gel\nрайон cit...,4500,"[\n {\n ""Title"": "" god of war: ragna..."
319,1072,god of war: ragnarok (ps4)\n100 gel\nрайон cit...,4800,"[\n {\n ""Title"": "" god of war: ragna..."
339,1072,god of war: ragnarok (ps4)\n100 gel\nрайон cit...,5100,"[\n {\n ""Title"": "" god of war: ragna..."
359,1072,god of war: ragnarok (ps4)\n100 gel\nрайон cit...,5400,"[\n {\n ""Title"": "" god of war: ragna..."


In [24]:
'\u0440\u0435\u0448\u0451\u0442\u043a\u0430 '.encode().decode()

'решётка '